In [3]:
from sklearn.externals import joblib
import os
import random
import numpy
import pandas as pd
from gensim.models import Doc2Vec
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.cross_validation import StratifiedKFold, cross_val_score
from sklearn.grid_search import GridSearchCV 

In [4]:
base_root='/home/yinqijin/tempdata/'

In [5]:
random.seed(666)

fin1  = open(base_root+'/training.csv','r')

pbin   = [ [],[],[],[],[] ]
nbin   = [ [],[],[],[],[] ]
bins   = []
neg    = []
pos    = []
for line in fin1:
	if line[0] == 'b':
		continue
	else:
		data  = line.split('"')
		if data[1] not in bins:
			bins.append(data[1])
		index = bins.index(data[1])
		data  = line.split(',')
		if data[7] == '1':
			pbin[index].append(line)
			pos.append(line)
		else:
			nbin[index].append(line)	


for i in range(len(bins)):
	nbinlen = len(nbin[i])
	temp    = random.sample(nbin[i],nbinlen/20)
	neg.extend(temp)

nnum   = len(neg)
pnum   = len(pos)
fnum   = len(neg[0].strip().split(',')) - 18

print nnum,pnum,fnum

1280 1291 60


In [6]:
fin2   = open(base_root+'enhancers.bed','r')
fin3   = open(base_root+'promoters.bed','r')
enhancers = []
promoters = []

arrays1 = numpy.zeros((nnum+pnum,100,2))
labels1 = numpy.zeros(nnum+pnum)

for line in fin2:
	data = line.strip().split()
	enhancers.append(data[3])

for line in fin3:
	data = line.strip().split()
	promoters.append(data[3])

enhancer_model = Doc2Vec.load(base_root+'enhancers_6_1_100.d2v')
promoter_model = Doc2Vec.load(base_root+'promoters_6_1_100.d2v')


for i in range(len(pos)):
        data = pos[i].strip().split(',')
        e_index = enhancers.index(data[5])
        p_index = promoters.index(data[10])
        prefix_enhancer = 'ENHANCERS_' + str(e_index)
        prefix_promoter = 'PROMOTERS_' + str(p_index)
        enhancer_vec = enhancer_model.docvecs[prefix_enhancer]
        promoter_vec = promoter_model.docvecs[prefix_promoter]
        enhancer_vec = enhancer_vec.reshape((100,1))
        promoter_vec = promoter_vec.reshape((100,1))
	arrays1[i] = numpy.append(enhancer_vec,promoter_vec,axis=1)
	labels1[i] = 1

for j in range(len(neg)):
        data = neg[j].strip().split(',')
        e_index = enhancers.index(data[5])
        p_index = promoters.index(data[10])
        prefix_enhancer = 'ENHANCERS_' + str(e_index)
        prefix_promoter = 'PROMOTERS_' + str(p_index)
        enhancer_vec = enhancer_model.docvecs[prefix_enhancer]
        promoter_vec = promoter_model.docvecs[prefix_promoter]
        enhancer_vec = enhancer_vec.reshape((100,1))
        promoter_vec = promoter_vec.reshape((100,1))  #.reshape((1,100))
	arrays1[i+j] = numpy.append(enhancer_vec,promoter_vec,axis=1)  #umn_stack
	labels1[i+j] = 0

In [7]:
print numpy.shape(arrays1)

(2571, 100, 2)


In [8]:
from numpy import random

In [9]:
index = range(len(labels1))
random.shuffle(index)
SPLIT_RATIO = 0.85
split_point =int(SPLIT_RATIO*len(labels1))


In [10]:
print split_point

2185


In [11]:
arrays1 = numpy.array(arrays1)
labels1 = numpy.array(labels1)

In [12]:
x_train , y_train=  arrays1[index[:split_point]] ,labels1[ index [:split_point]]
x_valid ,y_valid= arrays1[index[split_point:]] , labels1[index[split_point:]]

In [13]:
from keras.utils.np_utils import to_categorical

Using Theano backend.
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)


In [14]:
import pickle as pkl

In [15]:
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

In [16]:
savedata =open('./Gen_data/14_data.pkl','w')
pkl.dump(x_train,savedata)
pkl.dump(y_train,savedata)
pkl.dump(x_valid,savedata)
pkl.dump(y_valid,savedata)


In [2]:
loaddata = open('./Gen_data/14_data.pkl','r')
x_train=pkl.load(loaddata)
y_train=pkl.load(loaddata)
x_valid=pkl.load(loaddata)
y_valid=pkl.load(loaddata)

ValueError: insecure string pickle

In [17]:
import numpy
numpy.shape(y_train)

(2185, 2)

# Network

In [18]:
import numpy as np
import pandas as pd
import sys
import os

os.environ['KERAS_BACKEND']='theano'
from keras.engine.topology import Layer
from keras import initializations
from keras import backend as K

from collections  import defaultdict
import pandas as pd
import numpy as np

class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):


        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        '''
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [19]:
from keras.layers import Embedding
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout,Flatten

In [22]:
kmer_input = Input(shape=(100,2), dtype='float32')

l_lstm =Bidirectional(LSTM(20,return_sequences=True))(kmer_input)

l_lstm_drop = Dropout(0.3)(l_lstm)
l_att = AttLayer()(l_lstm_drop)
#l_flatten = Flatten()(l_lstm_drop)
l_att_drop = Dropout(0.5)(l_att)
#l_dense = Dense(256)(l_att_drop)
l_dense_1 = Dense(16)(l_att_drop)


preds = Dense(2, activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_dense_1)
model  = Model (kmer_input,preds)

In [23]:
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention LSTM network")
model.summary()


model fitting - attention LSTM network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 100, 2)        0                                            
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 100, 40)       3680        input_2[0][0]                    
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 100, 40)       0           bidirectional_2[0][0]            
____________________________________________________________________________________________________
attlayer_2 (AttLayer)            (None, 40)            40          dropout_3[0][0]                  
____________________________________________________

In [24]:
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), nb_epoch=100, batch_size=50)

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '135373' (I am process '7393')
INFO (theano.gof.compilelock): Waiting for existing lock by process '11960' (I am process '7393')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/yinqijin/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.2.1511-Core-x86_64-2.7.12-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '11960' (I am process '7393')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/yinqijin/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.2.1511-Core-x86_64-2.7.12-64/lock_dir


Train on 2185 samples, validate on 386 samples
Epoch 1/100
2185/2185 [==============================] - 7s - loss: 0.3747 - acc: 0.4975 - val_loss: 0.3739 - val_acc: 0.4404
Epoch 2/100
2185/2185 [==============================] - 7s - loss: 0.3745 - acc: 0.5112 - val_loss: 0.3730 - val_acc: 0.4378
Epoch 3/100
2185/2185 [==============================] - 7s - loss: 0.3742 - acc: 0.5176 - val_loss: 0.3720 - val_acc: 0.4585
Epoch 4/100
2185/2185 [==============================] - 7s - loss: 0.3744 - acc: 0.5039 - val_loss: 0.3734 - val_acc: 0.4534
Epoch 5/100
2185/2185 [==============================] - 7s - loss: 0.3742 - acc: 0.5272 - val_loss: 0.3726 - val_acc: 0.4637
Epoch 6/100
2185/2185 [==============================] - 7s - loss: 0.3740 - acc: 0.5204 - val_loss: 0.3715 - val_acc: 0.5233
Epoch 7/100
2185/2185 [==============================] - 7s - loss: 0.3737 - acc: 0.5327 - val_loss: 0.3702 - val_acc: 0.5544
Epoch 8/100
2185/2185 [==============================] - 7s - loss: 0.3

In [74]:
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), nb_epoch=80, batch_size=50)

Train on 2185 samples, validate on 386 samples
Epoch 1/80
2185/2185 [==============================] - 2s - loss: 0.3650 - acc: 0.6023 - val_loss: 0.3659 - val_acc: 0.5881
Epoch 2/80
2185/2185 [==============================] - 2s - loss: 0.3612 - acc: 0.6252 - val_loss: 0.3680 - val_acc: 0.5570
Epoch 3/80
2185/2185 [==============================] - 2s - loss: 0.3637 - acc: 0.5968 - val_loss: 0.3654 - val_acc: 0.5829
Epoch 4/80
2185/2185 [==============================] - 2s - loss: 0.3646 - acc: 0.5927 - val_loss: 0.3662 - val_acc: 0.5596
Epoch 5/80
2185/2185 [==============================] - 2s - loss: 0.3647 - acc: 0.6005 - val_loss: 0.3659 - val_acc: 0.5803
Epoch 6/80
2185/2185 [==============================] - 2s - loss: 0.3632 - acc: 0.6078 - val_loss: 0.3657 - val_acc: 0.5725
Epoch 7/80
2185/2185 [==============================] - 2s - loss: 0.3644 - acc: 0.5931 - val_loss: 0.3657 - val_acc: 0.5881
Epoch 8/80
2185/2185 [==============================] - 2s - loss: 0.3631 - ac

# Result  
1) 缺少两层全连接, 只有输出全连接层,100次,精度0.62  
2) 输出全连接层,缺少Att,100次,精度0.62   
3) 正常组: 加Att ,精度0.53  